## CPEX-CV Dropsonde Plotting Code (Questions?  Email brodenkirch@wisc.edu)

#### Given a directory that contains directories for each CPEX-CV research flight (that each contain QCed dropsonde CSV files, see _CPEX_CV_dropsonde.ipynb_), this code:

Calculates mean-layer metrics (e.g., RH, wind shear, CAPE, etc.) for each dropsonde using SHARPpy. The metrics for all dropsondes are then outputted to a CSV file (`Dropsonde_Metric_Calculations_CPEXCV_NEW.csv`) in the directory of this script **(NOTE:  some of these metrics purposefully are blank values, mostly because these blank metrics were meant to be manually inputted and/or were not useful/relevant anymore for the author)**.  The mid layer is defined as extending from the PBL top up to the freezing level.  The upper layer is defined as extending from the freezing level up to the _lowmax_hght_ **(you probably will want to change this!)**.  The deep layer is defined as extending from the near-surface (the given dropsonde's lowest available height level) up to the _lowmax_hght_.

Metrics that are calculated (in order of appearance on `Dropsonde_Metric_Calculations_CPEXCV_NEW.csv`) are:
* Sfc Pressure [mb]
* Sfc Height [m]
* PBL Top [mb] 
* PBL Top Height [m] 
* Freezing Level [mb] 
* Freezing Level Height [m]
* Upper Level Cap [mb] 
* Upper Level Cap Height [m]
* M10 Celsius Level [mb] 
* M20 Celsius Level [mb]
* M30 Celsius Level [mb]
* PBL RH [%]
* Mid Layer RH [%] 
* Upper Layer RH [%]
* Deep Layer RH [%]
* Below FZL MUCAPE [J/kg] 
* Above FZL MUCAPE [J/kg] 
* Deep Layer MUCAPE [J/kg] 
* MU CIN [J/kg]
* MU LCL [mb]
* MU EL [mb] 
* MU Max LI [degC] 
* MU Max LI Level [mb] 
* MU Max LI Layer **(always blank)**
* Below FZL MLCAPE [J/kg]
* Above FZL MLCAPE [J/kg]	
* Deep Layer MLCAPE [J/kg]
* ML CIN [J/kg]
* ML LCL [mb]
* ML EL [mb] 
* ML Max LI [degC]
* ML Max LI Level [mb] 
* ML Max LI Layer **(always blank)**
* SHARPpy Direct Method PBL Speed Shear [kts]	
* SHARPpy Direct Method PBL Directional Shear [deg] 
* SHARPpy Direct Method Mid Layer Speed Shear [kts] 
* SHARPpy Direct Method Mid Layer Directional Shear [deg] 
* SHARPpy Direct Method Upper Layer Speed Shear [kts]	
* SHARPpy Direct Method Upper Layer Directional Shear [deg]
* SHARPpy Direct Method Deep Layer Speed Shear [kts] 
* SHARPpy Direct Method Deep Layer Directional Shear [deg] 
* 10mb Interval Theta Gradient PBL Top [mb] **(always blank)**
* 10mb Interval Theta Gradient PBL RH [%] **(always blank)**
* 10mb Interval Theta Gradient Mid Layer RH [%] **(always blank)**
* Max Profile Height [m]
* 500m Bottom Cap Profile Height [m] 
* 500m Bottom Cap Deep Layer Speed Shear [kts] 
* 500m Bottom Cap Deep Layer Directional Shear [deg]

### Hope this helps!  Feel free to edit the code to fit your needs.  The variables you will definitely want to change right away are _day_folder_ and the _lowmax_hght_ parameter (cell #3).  Once you change these, everything should run properly as is.  If it doesn't, let the author know (brodenkirch@wisc.edu).

In [1]:
import os
import sys
#import xarray as xr
import pandas as pd
import numpy as np
#from datetime import datetime

#import sharppy
import sharppy.sharptab.profile as profile
#import sharppy.sharptab.interp as interp
import sharppy.sharptab.winds as winds
import sharppy.sharptab.utils as utils
import sharppy.sharptab.params as params
#import sharppy.sharptab.thermo as thermo

In [2]:
def calculate_dropsonde_metrics(drop_final_name, lowmax_hght):
    
    """ Calculate mean-layer metrics for dropsondes from a given CPEX-CV research flight using SHARPpy
    
    PARAMETERS
    ----------
    drop_final_name : the filepath (string) of the desired research flight's QCed dropsonde CSV file, which
                      was created using CPEXCV_dropsonde.ipynb
    
    lowmax_hght : the lowest maximum dropsonde height (in meters) of all dropsondes in the user's dataset
    
    
    RETURNS
    ----------
    No return variables, but the function adds the calculated dropsonde metrics from the desired research 
    flight to Dropsonde_Metric_Calculations_CPEXCV_NEW.csv, located in the directory of this script
    
    """
    
    drop_csv = pd.read_csv(drop_final_name)
    drop_times = drop_csv['Time [UTC]'].unique()  #sorted() = goes through files in alphabetical order

    for time in drop_times:
    #     print (time)
    #     if time == '2017-06-11 19:05:20':  #this dropsonde does not reach even close to the freezing level, which causes problems
    #         continue

        rel_data = drop_csv[drop_csv['Time [UTC]'] == time].copy()
        rel_data2 = rel_data.iloc[::-1]  #reverses the dataframe (row-based) to go from surface to upper-level

        pres = rel_data2['Pressure [mb]']
        hght = rel_data2['Height [m]']
        tmpc = rel_data2['Temperature [C]']
        dwpc = rel_data2['Dew Point [C]']
        wspd = 1.94384449 * rel_data2['Wind Speed [m/s]']  #converts m/s to knots (also in SHARPpy sharptab.utils script)
        wdir = rel_data2['Wind Direction [deg]']

        list_pres = pres.tolist()   
        list_hght = hght.tolist()
        list_wspd = wspd.tolist()
        list_wdir = wdir.tolist()

        try:
            prof = profile.create_profile(profile='default', pres=pres, hght=hght, tmpc=tmpc, dwpc=dwpc, wspd=wspd, wdir=wdir, missing=-9999, strictQC=True)
        except:  #if time == '2021-08-26 20:33:21'
            prof = profile.create_profile(profile='default', pres=pres, hght=hght, tmpc=tmpc, dwpc=dwpc, wspd=wspd, wdir=wdir, missing=-9999, strictQC=False)    

        sfc_p = prof.pres.data[prof.sfc]
        sfc_hght = prof.hght.data[prof.sfc]

        freeze_lev = params.temp_lvl(prof, temp = 0)
    #    print ('Profile Pressure Value at 0 C: %.1f' % freeze_lev)
        m10c_lev = params.temp_lvl(prof, temp = -10)
    #    print ('Profile Pressure Value at -10 C: %.1f' % m10c_lev)
        m20c_lev = params.temp_lvl(prof, temp = -20)
    #    print ('Profile Pressure Value at -20 C: %.1f' % m20c_lev)
        m30c_lev = params.temp_lvl(prof, temp = -30)
    #    print ('Profile Pressure Value at -30 C: %.1f' % m30c_lev)

        # Find the pressure/height value that corresponds closest to the lowest max dropsonde height without going over
        x = 0
        upper_lvl_hcap = -999.0  #just in case the dropsonde data doesn't reach the lowmax_hght when falling, this will alert you to skip the dropsonde
        while prof.hght.data[x] <= lowmax_hght:
            upper_lvl_pcap = prof.pres.data[x]
            upper_lvl_hcap = prof.hght.data[x]
            if x == (len(prof.hght.data) - 1):       #stops the loop if the highest (lowest) height (pressure) in the profile was just reached/assigned
                break
            else:
                x += 1
        #x = np.argmin(abs(prof.hght.data - lowmax_hght))
        #upper_lvl_pcap = prof.pres.data[x]
        #upper_lvl_hcap = prof.hght.data[x]
        if upper_lvl_hcap < 0:  #i.e., if the dropsonde data doesn't reach the lowmax_hght when falling
            print ('Bad dropsonde: Dropsonde at ', time, ' did not reach the lowmax_hght. \n')
            continue

        # Calculate PBL Top using the virtual potential temperature method (see Ajda's work and params.pbl_top code)
        pbl_top_pres = params.pbl_top(prof)
    #    print ('PBL Top: %.1f mb' % pbl_top_pres) # mb

        PBL_bot_index = 0  #i.e. list_pres.index(sfc_p)
        PBL_top_index = list_pres.index(pbl_top_pres)
        pbl_top_hght = prof.hght.data[PBL_top_index]

        mid_bot_index = PBL_top_index + 1
        upper_top_index = list_pres.index(upper_lvl_pcap)

        #Since freeze_lev isn't always a literal pressure entry in the profile, need to find the
            #lowest pressure in the mid layer (i.e., the pressure level closest to the freezing level without going over)
        z = 0
        freeze_lev_hght = -999.0  #just in case the dropsonde data doesn't reach the freezing level when falling, this will alert you to skip the dropsonde
        while prof.pres.data[z] >= freeze_lev:
            mid_top_index = z
            freeze_lev_hght = prof.hght.data[mid_top_index]
            z += 1
        upper_bot_index = z  #i.e. mid_top_index + 1
        #z = np.argmin(abs(prof.pres.data - freeze_lev))
        #mid_top_index = prof.pres.data[z]
        #upper_bot_index = prof.pres.data[z+1]
        if freeze_lev_hght < 0:  #i.e., if the dropsonde data doesn't reach the freezing level when falling
            print ('Bad dropsonde: Dropsonde at ', time, ' did not reach the freezing level. \n')
            continue

        print ('Working on metrics for dropsonde at:', time, '\n')
    #    print ('Dropsonde max height: %.1f m' % prof.hght.data[-1])
    #    print ('Surface Pressure: %.1f mb' % sfc_p)
    #    print ('Upper-level Pressure Cap: %.1f mb' % upper_lvl_pcap)
        
        drop_metrics = [''.join(time[:10].split('-')), time[11:].replace(':',''), '', '', '', '', '', '',
                       '', '', '', '', '', '', '', '', '', '']
        
        #mean-layer RH calculations based off of SHARPpy pressure threshold values (i.e. sfc - PBL Top, PBL Top - freezing level, freezing level - upper_lvl_pcap)
        PBL_RH_average = rel_data2[rel_data2['Pressure [mb]'] >= pbl_top_pres]['Relative Humidity [%]'].mean(skipna = True)
        mid_RH_average = rel_data2[(rel_data2['Pressure [mb]'] < pbl_top_pres) & (rel_data2['Pressure [mb]'] >= freeze_lev)]['Relative Humidity [%]'].mean(skipna = True)
        upper_RH_average = rel_data2[(rel_data2['Pressure [mb]'] < freeze_lev) & (rel_data2['Pressure [mb]'] >= upper_lvl_pcap)]['Relative Humidity [%]'].mean(skipna = True)
        deep_RH_average = rel_data2[rel_data2['Pressure [mb]'] >= upper_lvl_pcap]['Relative Humidity [%]'].mean(skipna = True)

    #     print ('PBL RH %:  ', np.round(PBL_RH_average,1))
    #     print ('Mid Layer RH %:  ', np.round(mid_RH_average,1))  
    #     print ('Upper Layer RH %:  ', np.round(upper_RH_average,1))   
    #     print ('Deep Layer RH %:  ', np.round(deep_RH_average,1))

        try:
            #mupcl = params.parcelx(prof, flag=3, exact = True) # Most-Unstable Parcel
            #mlpcl = params.parcelx(prof, flag=4, exact = True) # 100 mb Mean Layer Parcel (from sfc. to sfc. - 100mb)
            Dmupcl = params.parcelx(prof, flag=3, ptop = upper_lvl_pcap, exact = True) # Deep Layer Most-Unstable Parcel
            Bmupcl = params.parcelx(prof, flag=3, ptop = freeze_lev, exact = True) # Below Freezing Level Most-Unstable Parcel
            Dmlpcl = params.parcelx(prof, flag=4, ptop = upper_lvl_pcap, exact = True) # Deep layer 100 mb Mean Layer Parcel (from sfc. to sfc. - 100mb)
            Bmlpcl = params.parcelx(prof, flag=4, ptop = freeze_lev, exact = True) # Below Freezing Level 100 mb Mean Layer Parcel (from sfc. to sfc. - 100mb)    

            below_mucape = Bmupcl.bplus                #Below Freezing Level MUCAPE
            deep_mucape = Dmupcl.bplus                 #Deep Layer MUCAPE
            above_mucape = deep_mucape - below_mucape  #Above Freezing Level MUCAPE
            below_mlcape = Bmlpcl.bplus                #Below Freezing Level MLCAPE
            deep_mlcape = Dmlpcl.bplus                 #Deep Layer MLCAPE
            above_mlcape = deep_mlcape - below_mlcape  #Above Freezing Level MLCAPE
            no_moisture_data = False
        except:  #no moisture data for the dropsonde
            no_moisture_data = True

        #print ('Paste the following into Dropsonde_Metric_Calculations.csv:')
        #print ('')
        
        #Metrics independent of parcel type
        drop_metrics.append(np.round(sfc_p, 1))            
        drop_metrics.append(np.round(sfc_hght, 1))
        drop_metrics.append(np.round(pbl_top_pres, 1))
        drop_metrics.append(np.round(pbl_top_hght, 1))
        drop_metrics.append(np.round(freeze_lev, 1))
        drop_metrics.append(np.round(freeze_lev_hght, 1))
        drop_metrics.append(np.round(upper_lvl_pcap, 1))
        drop_metrics.append(np.round(upper_lvl_hcap, 1))
        drop_metrics.append(np.round(m10c_lev, 1))
        drop_metrics.append(np.round(m20c_lev, 1))
        drop_metrics.append(np.round(m30c_lev, 1))
        drop_metrics.append(np.round(PBL_RH_average, 1))
        drop_metrics.append(np.round(mid_RH_average, 1))
        drop_metrics.append(np.round(upper_RH_average, 1))   
        drop_metrics.append(np.round(deep_RH_average, 1))

        if no_moisture_data:            #if no thermodynamic instability metrics available     
            drop_metrics.append('')     #MU metrics start here    
            drop_metrics.append('')
            drop_metrics.append('')
            drop_metrics.append('')
            drop_metrics.append('')
            drop_metrics.append('')
            drop_metrics.append('')
            drop_metrics.append('')
            drop_metrics.append('')     #MU Max LI Layer (inputted manually if you want, though not used in analysis)
            drop_metrics.append('')     #ML metrics start here
            drop_metrics.append('')
            drop_metrics.append('')
            drop_metrics.append('')
            drop_metrics.append('')
            drop_metrics.append('')
            drop_metrics.append('')
            drop_metrics.append('')
            drop_metrics.append('')     #ML Max LI Layer (inputted manually if you want, though not used in analysis)
        else:    
            drop_metrics.append(np.round(below_mucape, 1))     #MU metrics start here     
            drop_metrics.append(np.round(above_mucape, 1))
            drop_metrics.append(np.round(deep_mucape, 1))
            drop_metrics.append(np.round(Dmupcl.bminus, 1))
            drop_metrics.append(np.round(Dmupcl.lclpres, 1))
            drop_metrics.append(np.round(Dmupcl.elpres, 1))
            drop_metrics.append(np.round(Dmupcl.limax, 2))
            drop_metrics.append(np.round(Dmupcl.limaxpres, 1))
            drop_metrics.append('')                            #MU Max LI Layer (inputted manually)
            drop_metrics.append(np.round(below_mlcape, 1))     #ML metrics start here
            drop_metrics.append(np.round(above_mlcape, 1))
            drop_metrics.append(np.round(deep_mlcape, 1))
            drop_metrics.append(np.round(Dmlpcl.bminus, 1))
            drop_metrics.append(np.round(Dmlpcl.lclpres, 1))
            drop_metrics.append(np.round(Dmlpcl.elpres, 1))
            drop_metrics.append(np.round(Dmlpcl.limax, 2))
            drop_metrics.append(np.round(Dmlpcl.limaxpres, 1))
            drop_metrics.append('')                            #ML Max LI Layer (inputted manually)

        #SHARPpy direct shear calculation method
        PBL_shear = winds.wind_shear(prof, pbot = sfc_p, ptop = pbl_top_pres)
        PBL_dir, PBL_speed = utils.comp2vec(PBL_shear[0], PBL_shear[1])
        mid_shear = winds.wind_shear(prof, pbot = list_pres[mid_bot_index], ptop = list_pres[mid_top_index])
        mid_dir, mid_speed = utils.comp2vec(mid_shear[0], mid_shear[1])
        upper_shear = winds.wind_shear(prof, pbot = list_pres[upper_bot_index], ptop = upper_lvl_pcap)
        upper_dir, upper_speed = utils.comp2vec(upper_shear[0], upper_shear[1])
        deep_shear = winds.wind_shear(prof, pbot = sfc_p, ptop = upper_lvl_pcap)
        deep_dir, deep_speed = utils.comp2vec(deep_shear[0], deep_shear[1])

        drop_metrics.append(np.round(PBL_speed, 2))
        drop_metrics.append(np.round(PBL_dir, 1))
        drop_metrics.append(np.round(mid_speed, 2))
        drop_metrics.append(np.round(mid_dir, 1))
        drop_metrics.append(np.round(upper_speed, 2))
        drop_metrics.append(np.round(upper_dir, 1))
        drop_metrics.append(np.round(deep_speed, 2))
        drop_metrics.append(np.round(deep_dir, 1))  

        #relevant theta-gradient mean-layer RH values (for PBL and mid-layer) (not really relevant anymore)
        drop_metrics.append('')  #10mb Interval Theta Gradient PBL Top [mb], if calculated (see download_files.py)
        drop_metrics.append('')  #10mb Interval Theta Gradient PBL RH [%], if calculated (see download_files.py)
        drop_metrics.append('')  #10mb Interval Theta Gradient Mid Layer RH [%], if calculated (see download_files.py)
        drop_metrics.append(prof.hght.data[-1])   #dropsonde max height for records

        #500m Bottom Cap Deep Layer Shear
        index500 = np.argmin(abs(prof.hght.data - 500))
        p500 = prof.pres.data[index500]
        h500 = prof.hght.data[index500]
        deep_500m_shear = winds.wind_shear(prof, pbot = p500, ptop = upper_lvl_pcap)
        deep_500m_dir, deep_500m_speed = utils.comp2vec(deep_500m_shear[0], deep_500m_shear[1])

        drop_metrics.append(np.round(h500, 1))    #dropsonde height closest to 500m (used for 500m - upper_lvl_cap deep shear calculation)
        drop_metrics.append(np.round(deep_500m_speed, 2))
        drop_metrics.append(np.round(deep_500m_dir, 1))
        print ('')
                    
        df_add = pd.DataFrame(data = [drop_metrics], columns = pd.read_csv('Dropsonde_Metric_Calculations_CPEXCV.csv').columns)
        
        #add the dropsonde metrics to Dropsonde_Metric_Calculations_CPEXCV_NEW.csv
        if os.path.isfile('Dropsonde_Metric_Calculations_CPEXCV_NEW.csv'):
            df_drop_original = pd.read_csv('Dropsonde_Metric_Calculations_CPEXCV_NEW.csv')
            df_drop_full = pd.concat([df_drop_original, df_add], ignore_index = True)  #concatenates fields with same header
            df_drop_full.to_csv('Dropsonde_Metric_Calculations_CPEXCV_NEW.csv', index = False)
        else: #first dropsonde of the first research flight, so need to create the new CSV file
            df_add.to_csv('Dropsonde_Metric_Calculations_CPEXCV_NEW.csv', index = False)
            

In [3]:
#for each CPEX-CV research flight, open the research flight's QCed dropsonde CSV file and 
    #calculate metrics for each dropsonde; metrics will be added into Dropsonde_Metric_Calculations_CPEXCV_NEW.csv
    
for x in sorted(os.listdir()):
    if os.path.isdir(x) and x[0:4] == '2022':
        if x != '20220916_prelim':
            day_folder = os.path.join(os.getcwd(), x)   
            drop_final_name = os.path.join(day_folder, 'final_dropsonde_' + x + '.csv')
            calculate_dropsonde_metrics(drop_final_name, lowmax_hght = 7622.5)
            
df_final = pd.read_csv('Dropsonde_Metric_Calculations_CPEXCV_NEW.csv')
df_final = df_final.replace(to_replace= '--', value = '')
df_final.to_csv('Dropsonde_Metric_Calculations_CPEXCV_NEW.csv', index = False)


Working on metrics for dropsonde at: 2022-09-06 11:33:43 



/Users/brodenkirch/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:1423: UserWarning: Warning: converting a masked element to nan.
  return interp_func(x, xp, fp, left, right)



Working on metrics for dropsonde at: 2022-09-06 13:33:39 



/Users/brodenkirch/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:1423: UserWarning: Warning: converting a masked element to nan.
  return interp_func(x, xp, fp, left, right)



Working on metrics for dropsonde at: 2022-09-06 13:40:21 



...

/Users/brodenkirch/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:1423: UserWarning: Warning: converting a masked element to nan.
  return interp_func(x, xp, fp, left, right)



Working on metrics for dropsonde at: 2022-09-30 14:13:27 



/Users/brodenkirch/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:1423: UserWarning: Warning: converting a masked element to nan.
  return interp_func(x, xp, fp, left, right)



Working on metrics for dropsonde at: 2022-09-30 14:25:20 



/Users/brodenkirch/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:1423: UserWarning: Warning: converting a masked element to nan.
  return interp_func(x, xp, fp, left, right)
